In [45]:
import requests as r
import re
import os
import logging
from datetime import datetime, timedelta
from dotenv import load_dotenv
import logging
from shapely import from_wkt
from shapely.geometry import Polygon, MultiPolygon

In [46]:
load_dotenv()

True

In [47]:
def get_access_token(client_id: str, client_secret: str) -> str:
    '''
    Supplies a temporary access token for of the OS NGD API
    Times out after 5 minutes
    Takes the project client_id and client_secret as input
    '''

    url = "https://api.os.uk/oauth2/token/v1"

    data = {
        "grant_type": "client_credentials"
    }

    response = r.post(
        url, 
        auth=(client_id, client_secret),
        data=data
    )

    json_response = response.json()
    if response.status_code >= 400:
        raise Exception(json_response)
    token = json_response["access_token"]

    return token

In [48]:
def OAauth2Manager(func: callable):

    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception:
            client_id = os.environ.get('CLIENT_ID')
            client_secret = os.environ.get('CLIENT_SECRET')
            access_token = get_access_token(
                client_id=client_id,
                client_secret=client_secret
            )
            kwargs['access_token'] = access_token
            return func(*args, **kwargs)

    return wrapper

In [49]:
def generic_request(url: str, headers: dict = {}, access_token: str = None, **kwargs) -> dict:

    if access_token:
        headers['Authorization'] = f"Bearer {access_token}"
    response = r.get(url, headers=headers, **kwargs)
    json_response = response.json()
    if response.status_code >= 400:
        raise Exception(json_response)

    return json_response

In [50]:
def construct_query_params(**kwargs) -> str:
    '''
    Constructs a query string from a dictionary of key-value pairs.
    Refer to https://osdatahub.os.uk/docs/ofa/technicalSpecification for details about query parameters.
    The options are:
        - bbox
        - bbox-crs
        - crs
        - datetime
        - filter
        - filter-crs
        - filter-lang
        - limit
        - offset
    '''
    params_list = [f'{k}={v}' for k, v in kwargs.items()]
    return '?' + '&'.join(params_list)

In [51]:
def wkt_to_spatial_filter(wkt):
    return f'(INTERSECTS(geometry,{wkt}))'

In [52]:
def construct_filter_param(**kwargs):
    for k, v in kwargs.items():
        if type(v) == str:
            kwargs[k] = f"'{v}'"
    filter_list = [f"({k}={v})" for k, v in kwargs.items()]
    return 'and'.join(filter_list)

In [53]:
def ngd_items_request(
    collection: str,
    query_params: dict = {},
    filter_params: dict = {},
    headers: dict = {},
    access_token: str = None,
    **kwargs
) -> dict:
    
    filter_params = filter_params.copy()
    query_params = query_params.copy()

    if filter_params:
        filters = construct_filter_param(**filter_params)
        current_filters = query_params.get('filter')
        query_params['filter'] = f'({current_filters})and{filters}' if current_filters else filters

    wkt = kwargs.get('wkt')
    if wkt:
        del kwargs['wkt']
        spatial_filter = wkt_to_spatial_filter(wkt)
        current_filters = query_params.get('filter')
        query_params['filter'] = f'({current_filters})and{spatial_filter}' if current_filters else spatial_filter


    query_params_string = construct_query_params(**query_params)
    url = f'https://api.os.uk/features/ngd/ofa/v1/collections/{collection}/items/{query_params_string}'
    print(url)

    if access_token:
        headers['Authorization'] = f"Bearer {access_token}"
    response = r.get(url, headers=headers, **kwargs)
    json_response = response.json()

    if response.status_code >= 400:
        raise Exception(json_response)

    return json_response

In [54]:
def get_latest_collection_versions(flag_recent_updates: bool = True, recent_update_days: int = 31) -> tuple[dict[str: str], list[str]]:
    '''
    Returns the latest collection versions of each NGD collection.
    Feature collections follow the following naming convention: theme-collection-featuretype-version (eg. bld-fts-buildingline-2)
    The output of this function maps base feature collection names (theme-collection-featuretype) to the full name, including the latest version.
    This can be used to ensure that software is always using the latest version of a feature collection.
    More details on feature collection naming can be found at https://docs.os.uk/osngd/accessing-os-ngd/access-the-os-ngd-api/os-ngd-api-features/what-data-is-available
    '''
    response = r.get('https://api.os.uk/features/ngd/ofa/v1/collections/')
    collections_data = response.json()['collections']
    collections_list = [collection['id'] for collection in collections_data]
    collection_base_names = set([re.sub(r'-\d+$', '', c) for c in collections_list])
    output_lookup = dict()
    for base_name in collection_base_names:
        all_versions = [c for c in collections_list if c.startswith(base_name)]
        latest_version = max(all_versions, key=lambda c: int(c.split('-')[-1]))
        output_lookup[base_name] = latest_version
    recent_collections = None
    if flag_recent_updates:
        time_format = r'%Y-%m-%dT%H:%M:%SZ'
        recent_update_cutoff = datetime.now() - timedelta(days=recent_update_days)
        latest_versions_data = [c for c in collections_data if c['id'] in output_lookup.values()]
        recent_collections = list()
        for collection_data in latest_versions_data:
            version_startdate = collection_data['extent']['temporal']['interval'][0][0]
            time_obj = datetime.strptime(version_startdate, time_format)
            if time_obj > recent_update_cutoff:
                collection = collection_data['id']
                recent_collections.append(collection)
                logging.warning(f'{collection} is a recent version/update from the last {recent_update_days} days.')
    return output_lookup, recent_collections

In [55]:
def get_single_latest_collection(collection: str, **kwargs) -> str:
    '''
    Returns the latest collection of a given collection base.
    Input must be in the format theme-collection-featuretype (eg. bld-fts-buildingline)
    Output will complete the full name of the feature collection by appending the latest version number (eg. bld-fts-buildingline-2)
    More details on feature collection naming can be found at https://docs.os.uk/osngd/accessing-os-ngd/access-the-os-ngd-api/os-ngd-api-features/what-data-is-available
    '''
    latest_collections = get_latest_collection_versions(**kwargs)
    latest_collection = latest_collections[collection]
    return latest_collection

In [56]:
def multiple_collections_extension(func: callable) -> dict:

    def wrapper(*args, collections: list[str], **kwargs):
        results = dict()
        for c in collections:
            json_response = func(c, **kwargs)
            results[c] = json_response
        return results

    return wrapper

In [ ]:
def feature_limit_extension(func: callable):
    f"""
    This is an extension of functions returning ngd features. It serves to extend the maximum number of features returned above the default maximum 100 by looping through multiple requests.
    It takes the following arguments:
    - collection: The name of the collection to be queried.
    - request_limit: The maximum number of requests to be made function. Default is 50.
    - feature_limit: The maximum number of features to be returned. Default is None.
    - query_params: A dictionary of query parameters to be passed to the function. Default is an empty dictionary.
    To prevent indefinite requests and high costs, at least one of feature_limit or request_limit must be provided, although there is no limit to the upper value these can be.
    It will make multiple requests to the function to compile all features from the specified collection, returning a dictionary with the features and metadata.
    """
    def wrapper(
        *args,
        request_limit: int = 50,
        feature_limit: int = None,
        query_params: dict = {},
        **kwargs
    ):

        query_params = query_params.copy()
        if 'offset' in query_params:
            raise AttributeError('offset is not a valid argument for functions using this decorator.')

        items = list()

        batch_count, final_batchsize = divmod(feature_limit, 100) if feature_limit else (None, None)
        request_count = 0
        offset = 0

        if not(feature_limit) and not(request_limit):
            raise AttributeError('At least one of feature_limit or request_limit must be provided to prevent indefinitely numerous requests and high costs. However, there is no upper limit to these values.')

        while (request_count != request_limit) and (not(feature_limit) or offset < feature_limit):

            if request_count == batch_count:
                print('final batch of size', final_batchsize)
                query_params['limit'] = final_batchsize
            query_params['offset'] = offset

            json_response = func(*args, query_params=query_params, **kwargs)
            items += json_response['features']

            if not [link for link in json_response['links'] if link['rel'] == 'next']:
                break

            request_count += 1
            offset += 100

        geojson = {
            "type": "FeatureCollection",
            "numberOfRequests": request_count + 1,
            "totalNumberReturned": len(items),
            "timeStamp": datetime.now().isoformat(),
            "collection": kwargs.get('collection'),
            "source": "Compiled from code by Geovation from Ordnance Survey",
            "features": items
        }
        return geojson

    return wrapper

In [ ]:
def multigeometry_search_extension(func: callable):

    def wrapper(*args, wkt: str, **kwargs):

        multi_geom = from_wkt(wkt) if type(wkt) == str else wkt
        search_areas = list()

        for search_area, geom in enumerate(multi_geom.geoms):
            json_response = func(*args, wkt=geom, **kwargs)
            json_response['searchAreaNumber'] = search_area
            search_areas.append(json_response)

        geojson = {
            "type": "FeatureCollection",
            "searchAreas": search_areas
        }

        return geojson

    return wrapper

In [59]:
x=Polygon({(400000,400000), (400050,400000), (400050,400050), (400000,40050)})
y=Polygon({(500000,500000), (500050,500000), (500050,500050), (500000,50050)})

In [60]:
z=MultiPolygon([x,y])

# Example Ways of Building Custom APIs by Combining Wrappers

In [121]:
ngd_items_request_OAuth = OAauth2Manager(ngd_items_request)

In [120]:
ngd_items_OAuth_multigeom = multigeometry_search_extension(ngd_items_request_OAuth)
ngd_items_OAuth_all_features = feature_limit_extension(ngd_items_request_OAuth)

In [129]:
ngd_items_mulitgeom_all_feats = multigeometry_search_extension(ngd_items_OAuth_all_features )

In [115]:
ngd_items_OAuth_all_features(
    request_limit=3,
    collection='bld-fts-buildingline-1',
    query_params={
        'crs':'http://www.opengis.net/def/crs/EPSG/0/27700',
        'filter':'''(INTERSECTS(geometry,MULTIPOLYGON (((400000 400000, 400050 400050, 400050 400000, 400000 40050, 400000 400000)), ((500050 500050, 500050 500000, 500000 500000, 500000 50050, 500050 500050)))))''',
        'filter-crs':'http://www.opengis.net/def/crs/EPSG/0/27700'
    })

https://api.os.uk/features/ngd/ofa/v1/collections/bld-fts-buildingline-1/items/?crs=http://www.opengis.net/def/crs/EPSG/0/27700&filter=(INTERSECTS(geometry,MULTIPOLYGON (((400000 400000, 400050 400050, 400050 400000, 400000 40050, 400000 400000)), ((500050 500050, 500050 500000, 500000 500000, 500000 50050, 500050 500050)))))&filter-crs=http://www.opengis.net/def/crs/EPSG/0/27700&offset=0
https://api.os.uk/features/ngd/ofa/v1/collections/bld-fts-buildingline-1/items/?crs=http://www.opengis.net/def/crs/EPSG/0/27700&filter=(INTERSECTS(geometry,MULTIPOLYGON (((400000 400000, 400050 400050, 400050 400000, 400000 40050, 400000 400000)), ((500050 500050, 500050 500000, 500000 500000, 500000 50050, 500050 500050)))))&filter-crs=http://www.opengis.net/def/crs/EPSG/0/27700&offset=0
https://api.os.uk/features/ngd/ofa/v1/collections/bld-fts-buildingline-1/items/?crs=http://www.opengis.net/def/crs/EPSG/0/27700&filter=(INTERSECTS(geometry,MULTIPOLYGON (((400000 400000, 400050 400050, 400050 400000,

{'type': 'FeatureCollection',
 'numberOfRequests': 3,
 'totalNumberReturned': 300,
 'timeStamp': '2024-12-09T12:35:41.238597',
 'collection': 'bld-fts-buildingline-1',
 'source': 'Compiled from code by Geovation from Ordnance Survey',
 'features': [{'id': '00086ca4-b130-4c72-89cd-2ad67d5c91d1',
   'type': 'Feature',
   'geometry': {'type': 'LineString',
    'coordinates': [[500006.3, 153090.4],
     [500008.3, 153087.5],
     [500009.2, 153088.4]]},
   'properties': {'osid': '00086ca4-b130-4c72-89cd-2ad67d5c91d1',
    'toid': 'osgb1000001790316969',
    'theme': 'Buildings',
    'changetype': 'New',
    'isobscured': False,
    'description': 'Overhanging Building Edge',
    'versiondate': '2022-08-26',
    'physicallevel': 'Level 1',
    'geometry_length': 4.795575,
    'geometry_source': 'Ordnance Survey',
    'description_source': 'Ordnance Survey',
    'geometry_updatedate': '2005-03-22',
    'capturespecification': 'Rural',
    'geometry_evidencedate': '2005-03-22',
    'descripti

In [ ]:
multi

## Demos

In [66]:
ngd_items_OAuth_all_features('bld-fts-buildingline-1', wkt=x, query_params={'filter-crs':'http://www.opengis.net/def/crs/EPSG/0/27700'}, request_limit=4)

https://api.os.uk/features/ngd/ofa/v1/collections/bld-fts-buildingline-1/items/?filter-crs=http://www.opengis.net/def/crs/EPSG/0/27700&offset=0&filter=(INTERSECTS(geometry,POLYGON ((400000 400000, 400050 400050, 400050 400000, 400000 40050, 400000 400000))))
https://api.os.uk/features/ngd/ofa/v1/collections/bld-fts-buildingline-1/items/?filter-crs=http://www.opengis.net/def/crs/EPSG/0/27700&offset=0&filter=(INTERSECTS(geometry,POLYGON ((400000 400000, 400050 400050, 400050 400000, 400000 40050, 400000 400000))))
https://api.os.uk/features/ngd/ofa/v1/collections/bld-fts-buildingline-1/items/?filter-crs=http://www.opengis.net/def/crs/EPSG/0/27700&offset=100&filter=(INTERSECTS(geometry,POLYGON ((400000 400000, 400050 400050, 400050 400000, 400000 40050, 400000 400000))))
https://api.os.uk/features/ngd/ofa/v1/collections/bld-fts-buildingline-1/items/?filter-crs=http://www.opengis.net/def/crs/EPSG/0/27700&offset=200&filter=(INTERSECTS(geometry,POLYGON ((400000 400000, 400050 400050, 400050 

{'type': 'FeatureCollection',
 'numberOfRequests': 4,
 'totalNumberReturned': 400,
 'timeStamp': '2024-12-09T11:46:20.312087',
 'collection': None,
 'source': 'Compiled from code by Geovation from Ordnance Survey',
 'features': [{'id': '000d881d-fffe-448f-b7a4-75e43ef47731',
   'type': 'Feature',
   'geometry': {'type': 'LineString',
    'coordinates': [[-2.0010176, 52.8283301], [-2.0010143, 52.8283639]]},
   'properties': {'osid': '000d881d-fffe-448f-b7a4-75e43ef47731',
    'toid': 'osgb5000005268285706',
    'theme': 'Buildings',
    'changetype': 'New',
    'isobscured': False,
    'description': 'Building Internal Division',
    'versiondate': '2022-08-26',
    'physicallevel': 'Surface Level',
    'geometry_length': 3.76555,
    'geometry_source': 'Ordnance Survey',
    'description_source': 'Ordnance Survey',
    'geometry_updatedate': '2020-08-12',
    'capturespecification': 'Rural',
    'geometry_evidencedate': '2020-04-07',
    'description_updatedate': '2020-08-13',
    'ver

In [ ]:
import geopandas as gpd

  Using cached geopandas-1.0.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached pyogrio-0.10.0-cp312-cp312-macosx_12_0_x86_64.whl.metadata (5.5 kB)
  Using cached pandas-2.2.3-cp312-cp312-macosx_10_9_x86_64.whl.metadata (89 kB)
  Using cached pyproj-3.7.0-cp312-cp312-macosx_12_0_x86_64.whl.metadata (31 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached geopandas-1.0.1-py3-none-any.whl (323 kB)
Using cached pandas-2.2.3-cp312-cp312-macosx_10_9_x86_64.whl (12.5 MB)
Using cached pyogrio-0.10.0-cp312-cp312-macosx_12_0_x86_64.whl (16.4 MB)
Using cached pyproj-3.7.0-cp312-cp312-macosx_12_0_x86_64.whl (6.3 MB)
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)


In [110]:
test_geom = gpd.read_file('Data/MultiPolygon1.shp', crs=27700)

/Users/mchapman/Documents/CatalyST/.venv/lib/python3.12/site-packages/pyogrio/raw.py:198: RuntimeWarning: driver ESRI Shapefile does not support open option CRS
  return ogr_read(


In [111]:
test_geom = test_geom.geometry.unary_union

/var/folders/sg/3csqph_x7kz5xx50dz_k8cyx5wdl6y/T/ipykernel_44186/104286071.py:1: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  test_geom = test_geom.geometry.unary_union


In [112]:
from shapely import set_precision

In [113]:
print(set_precision(test_geom,0))

MULTIPOLYGON (((521796.0390856008 180096.86936259994, 521822.1546492428 179921.65215072298, 521662.4245739442 179891.58888560024, 521620.5182043792 180086.24093553636, 521711.9226771261 180091.4033143958, 521796.0390856008 180096.86936259994)), ((625109.512522613 309945.5774759307, 625138.6647797017 309932.21602476505, 624994.7255103262 309773.7006268451, 624695.9148751667 309810.140948206, 624712.3130197792 309898.2050581615, 624938.8503509061 309906.10046112305, 624933.9916413913 309923.71328311414, 625109.512522613 309945.5774759307)), ((340810.9316758898 498679.7192230167, 340670.02909996104 498655.4256754428, 340662.74103568884 498830.3392179751, 340784.2087735585 498818.1924441881, 340810.9316758898 498679.7192230167)))


In [109]:
print(set_precision(test_geom,0))

MULTIPOLYGON (((521796.0390856008 180096.86936259994, 521822.1546492428 179921.65215072298, 521662.4245739442 179891.58888560024, 521620.5182043792 180086.24093553636, 521711.9226771261 180091.4033143958, 521796.0390856008 180096.86936259994)), ((625109.512522613 309945.5774759307, 625138.6647797017 309932.21602476505, 624994.7255103262 309773.7006268451, 624695.9148751667 309810.140948206, 624712.3130197792 309898.2050581615, 624938.8503509061 309906.10046112305, 624933.9916413913 309923.71328311414, 625109.512522613 309945.5774759307)), ((340810.9316758898 498679.7192230167, 340670.02909996104 498655.4256754428, 340662.74103568884 498830.3392179751, 340784.2087735585 498818.1924441881, 340810.9316758898 498679.7192230167)))


In [132]:
data=ngd_items_mulitgeom_all_feats('lnd-fts-land-3', wkt=set_precision(test_geom,0), query_params={'filter-crs':'http://www.opengis.net/def/crs/EPSG/0/27700'})

https://api.os.uk/features/ngd/ofa/v1/collections/lnd-fts-land-3/items/?filter-crs=http://www.opengis.net/def/crs/EPSG/0/27700&offset=0&filter=(INTERSECTS(geometry,POLYGON ((521796.0390856008 180096.86936259994, 521822.1546492428 179921.65215072298, 521662.4245739442 179891.58888560024, 521620.5182043792 180086.24093553636, 521711.9226771261 180091.4033143958, 521796.0390856008 180096.86936259994))))
https://api.os.uk/features/ngd/ofa/v1/collections/lnd-fts-land-3/items/?filter-crs=http://www.opengis.net/def/crs/EPSG/0/27700&offset=0&filter=(INTERSECTS(geometry,POLYGON ((521796.0390856008 180096.86936259994, 521822.1546492428 179921.65215072298, 521662.4245739442 179891.58888560024, 521620.5182043792 180086.24093553636, 521711.9226771261 180091.4033143958, 521796.0390856008 180096.86936259994))))
https://api.os.uk/features/ngd/ofa/v1/collections/lnd-fts-land-3/items/?filter-crs=http://www.opengis.net/def/crs/EPSG/0/27700&offset=100&filter=(INTERSECTS(geometry,POLYGON ((521796.039085600

In [145]:
data

{'type': 'FeatureCollection',
 'searchAreas': [{'type': 'FeatureCollection',
   'numberOfRequests': 2,
   'totalNumberReturned': 238,
   'timeStamp': '2024-12-09T12:45:54.390758',
   'collection': None,
   'source': 'Compiled from code by Geovation from Ordnance Survey',
   'features': [{'id': '0099c227-9e4d-49b6-a3bd-5347330bb481',
     'type': 'Feature',
     'geometry': {'type': 'Polygon',
      'coordinates': [[[-0.2472766, 51.5060218],
        [-0.2471184, 51.5060451],
        [-0.2471388, 51.5061016],
        [-0.2471859, 51.5060946],
        [-0.2471662, 51.5060449],
        [-0.2472648, 51.5060297],
        [-0.2472766, 51.5060218]]]},
     'properties': {'osid': '0099c227-9e4d-49b6-a3bd-5347330bb481',
      'toid': 'osgb1000001787219179',
      'theme': 'Land',
      'status': None,
      'istidal': False,
      'landform': None,
      'nlud_code': 'U071',
      'changetype': 'New',
      'isobscured': False,
      'description': 'Residential Garden',
      'versiondate': '202